In [2]:
def reverseComplement(s):
    complement = {'A':'T', 'T':'A', 'G':'C','C':'G', 'N':'N'}
    t=''
    for base in s:
        t = complement[base] + t
    return t
reverseComplement('GTGTATAGGATT')

'AATCCTATACAC'

In [3]:
def readGenome(filename):
    genome=''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

'GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC'

In [30]:
def readFastq(filename):
    sequences =[]
    qualities=[]
    with open(filename) as fh:
        while True:
            fh.readline()                 # Line 1: header → not stored, # Skip the header (@...)
            seq = fh.readline().rstrip()  # Line 2: sequence → store in `sequences`
            fh.readline()                 # Line 3: plus sign → not stored
            qual = fh.readline().rstrip() # Line 4: quality → store in `qualities`, # Skip the plus line (+)
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [31]:
def naive(p, t):                         # pattern and text; p = x and t = y
    occurrences = []                     # track all indexes p matches agains t
    for i in range(len(t) - len(p) + 1): # Loop over alignments
        match = True
        for j in range(len(p)):          # Loop over characters of the pattern (1 can match or more)
            if t[i+j] != p[j]:           # compare characters
                match = False            # mismatch; reject alignment
                break
        if match:                        # What would go wrong if you moved if match: inside the loop? 
                                         # This would cause occurrences.append(i) to run after the first character matches, 
                                         # not after all characters match
            occurrences.append(i)        # alL chars matched; record
    return occurrences

# How many possible alignments? y-x+1
# How many number of characters possible comparison? x(y-x+1)
# Least number of characters possible? y-x+1 (first character can be a mismatch)

#### Question 1

How many times does AGGT or its reverse complement (ACCT) occur in the lambda virus genome? E.g. if AGGT
occurs 10 times and ACCT occurs 12 times, you should report 22.

In [ ]:
# 1
genome = readGenome('lambda_virus.fa')
p = 'AGGT'
rev_p = reverseComplement(p)

matches1 = naive(p,genome)
matches2 = naive(rev_p, genome)

print(len(matches1)+len(matches2))

#### Question 2

How many times does TTAA or its reverse complement occur in the lambda virus genome?

Hint: TTAA and its reverse complement are equal, so remember not to double count.

In [64]:
# 2
genome = readGenome('lambda_virus.fa')
p = 'TTAA'
matches1 = naive(p,genome)

print(len(matches1))

195


#### Question 3

What is the offset of the leftmost occurrence of ACTAAGT or its reverse complement in the Lambda virus
genome? E.g. if the leftmost occurrence of ACTAAGT is at offset 40 (0-based) and the leftmost occurrence of its
reverse complement ACTTAGT is at offset 29, then report 29.

In [65]:
# 3
p = 'ACTAAGT'
rev_p = reverseComplement(p)
matches1 = naive(p,genome)
matches2 = naive(rev_p, genome)

if matches1 and matches2:
    offset = min(matches1[0], matches2[0])
elif matches1:
    offset = matches1[0]
elif matches2:
    offset = matches2[0]
else:
    offset = -1  # not found

print("Correct offset:", offset)

Correct offset: 26028


#### Question 4

What is the offset of the leftmost occurrence of AGTCGA or its reverse complement in the Lambda virus
genome?

In [67]:
# 4
p = 'AGTCGA'
rev_p = reverseComplement(p)
matches1 = naive(p,genome)
matches2 = naive(rev_p, genome)

if matches1 and matches2:
    print(min(matches1[0], matches2[0]))
elif matches1:
    print(matches1[0])
elif matches2:
    print(matches2[0])
else:
    print("Not found")

450


#### Question 5

As we will discuss, sometimes we would like to find approximate matches for Pin T. That is, we want to find
occurrences with one or more differences.

For Questions 5 and 6, make a new version of the naive function called naive_2mm that allows up to 2
mismatches per occurrence. Unlike for the previous questions, do not consider the reverse complement here.
We're looking for approximate matches for Pitself, not its reverse complement.

For example, ACTTTA occurs twice in ACTTACTTGATAAAGT, once at offset 0 with 2 mismatches, and once at
offset 4 with 1 mismatch. So naive_2mm (' ACTTTA' , 'ACTTACTTGATAAAGT' ) should return the list
[o, 4].

Hint: See this notebook for a few examples you can use to test your nai ve_2mm function.

How many times does TTCAAGCC occur in the Lambda virus genome when allowing up to 2 mismatches?

In [9]:
def naive_2mm(p, t):                         # pattern and text; p = x and t = y
    occurrences = []                     # track all indexes p matches agains t
    for i in range(len(t) - len(p) + 1): # Loop over alignments
        match = True
        mismatches = 0
        for j in range(len(p)):          # Loop over characters of the pattern (1 can match or more)
            if t[i+j] != p[j]: 
                mismatches += 1 
                if mismatches > 2:
                    match = False
                    break
        if match:                  
            occurrences.append(i)        # alL chars matched; record
    return occurrences

In [68]:
# 5
p = 'TTCAAGCC'
matches1 = naive_2mm(p,genome)
print(len(matches1))

191


#### Question 6

What is the offset of the leftmost occurrence of AGGAGGTT in the Lambda virus genome when allowing up to 2
mismatches?

In [69]:
# 6
p = 'AGGAGGTT'
matches1 = naive_2mm(p,genome)
print(min(matches1))

49


#### Question 7

Finally, download and parse the provided FASTQ file containing real DNA sequencing reads derived from a
human:

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900 1.first1000.fastq

Note that the file has manyreads in it and you should examine all of them together when answering this
question. The reads are taken from this study:

Ajay, S. S., Parker, S. C., Abaan, H. O., Fajardo, K. V. F., & Margulies, E. H. (2011). Accurate

and comprehensive sequencing of personal genomes. Genome research, 21(9), 1498-1505.

This dataset has something wrong with it; one of the sequencing cycles is poor quality.

Report which sequencing cycle has the problem. Remember that a sequencing cycle corresponds to a
particular offset in all the reads. For example, if the leftmost read position seems to have a problem consistently
across reads, report 0. If the fourth position from the left has the problem, report 3. Do whatever analysis you
think is needed to identify the bad cycle. It might help to review the "Analyzing reads by position" video.

In [32]:
seqs, quals = readFastq('ERR037900_1.first1000.fastq')

In [23]:
def phred33ToQ(qual):
    """Turn Phred+33 ASCII-encoded quality into Q"""
    return ord(qual)-33

In [22]:
[print(q[:30]) for q in quals]

HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHIHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHIFHFEGGF
HHFHHDHHHHDDGGGDHDHHHHHGHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHIHHHHHHHHHH
HHHGHGHHGHHHHHHHHHHHHHHHHHEHHH
HHHHHHHGHHHHHHHHHHHFHHHHHFHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHFHHHEHHHHHEG
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHEHH
HHHHHHHHHHHHHHHHDHHHHHHHGHHHHH
HHHHHHHHHHHHHGHHHHHHHHIHHHGHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHIHHHHHIHHHHGHHHHHGHGHHHCH
HHHHHHHHHHHHHHHFDGGGHHHHHDFD@F
HHHHHHHHHHHHHHGHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHGHHHFHHH
HHHHGHHHHHHHHHHHHHHHHHHHH4DBFF
HHHHHHHHHHHHIHHHHHHHHHHHFHHHHH
HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHH

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

### Explaining the exercise

If the leftmost read position seems to have a problem consistently across reads, report 0. If the fourth position from the left has the problem, report 3:
Analyze average quality score by position (aka cycle) across all reads, and return the index with the worst average. "The leftmost read" is just the first character of a string, so index 0, and the 3 is the fourth character in the string. It just says to report the index of the worst cycle according to which read.

Each read has a corresponding quality string, one character per base, indicating the confidence of the base call at each position (also called cycle).

So, what does "cycle" mean?
Cycle 0 = position 0 (first base in every read)

Cycle 1 = position 1 (second base in every read)

Cycle 3 = position 3 (fourth base in every read)

If the average quality is consistently low at position 0 across all reads → "report 0".
If it’s low at position 3 → "report 3".

The final answer is just the index (position) of the worst-quality cycle.

In [61]:
def find_bad_cycle(qualities):
    n_cycles = len(qualities[0])
    total_scores = [0] * n_cycles
    for qual in qualities:
        for i in range(n_cycles):
            total_scores[i] += phred33ToQ(qual[i])
    avg_scores = [total / len(qualities) for total in total_scores]
    return avg_scores.index(min(avg_scores))  # bad cycle = lowest avg score

#### Result
In this case it’s low at position 66 → "report 66".

#### clarity, insight, and debug output

In [60]:
def find_bad_cycle(qualities):
    n_cycles = len(qualities[0])
    total_scores = [0] * n_cycles
    
    for qual in qualities:
        for i in range(n_cycles):
            total_scores[i] += phred33ToQ(qual[i])
    
    avg_scores = [total / len(qualities) for total in total_scores]

    # Print average scores to see what's going on
    for i, score in enumerate(avg_scores):
        print(f"Cycle {i}: Average quality = {score:.2f}")
    
    worst_cycle = avg_scores.index(min(avg_scores))
    print(f"\nWorst cycle is {worst_cycle} (lowest average quality).")
    return worst_cycle
    
bad_cycle = find_bad_cycle(quals)
print(f"Bad sequencing cycle: {bad_cycle}")

Cycle 0: Average quality = 38.44
Cycle 1: Average quality = 38.46
Cycle 2: Average quality = 38.40
Cycle 3: Average quality = 38.43
Cycle 4: Average quality = 38.42
Cycle 5: Average quality = 38.38
Cycle 6: Average quality = 38.36
Cycle 7: Average quality = 38.36
Cycle 8: Average quality = 38.33
Cycle 9: Average quality = 38.34
Cycle 10: Average quality = 38.19
Cycle 11: Average quality = 38.15
Cycle 12: Average quality = 38.09
Cycle 13: Average quality = 38.15
Cycle 14: Average quality = 38.15
Cycle 15: Average quality = 38.10
Cycle 16: Average quality = 38.05
Cycle 17: Average quality = 38.04
Cycle 18: Average quality = 38.07
Cycle 19: Average quality = 38.15
Cycle 20: Average quality = 38.09
Cycle 21: Average quality = 37.99
Cycle 22: Average quality = 38.01
Cycle 23: Average quality = 37.91
Cycle 24: Average quality = 38.03
Cycle 25: Average quality = 37.58
Cycle 26: Average quality = 37.51
Cycle 27: Average quality = 37.50
Cycle 28: Average quality = 37.58
Cycle 29: Average qualit